In [1]:
import numpy as np
import pandas as pd
# import sklearn.model_selection
# from sklearn.model_selection import train_test_split
# from sklearn.linear_model import LogisticRegression
# from sklearn.metrics import accuracy_score
# from sklearn.metrics import confusion_matrix
# from sklearn.metrics import classification_report
# from sklearn import preprocessing
# from sklearn.feature_selection import RFE
# from sklearn.ensemble import RandomForestClassifier
import pickle
from IPython.core.display import HTML
#import matplotlib
# import seaborn as sns
# import matplotlib.pyplot as plt
# from datetime import datetime
# from collections import OrderedDict
%matplotlib inline

<div class="span5 alert alert-info">
<h2>Notebook details</h2>

<p> This notebook is for <b>Cluesting</b> for <b>Mortagage custome segementation</b> project.</p>

<p> Notes.</p>
<ol>
<li>Perform baseline analysis </li>
<li>Perform feature important analysis </li>
<li>Perform multiple runs for training and test data to understand variation in accuracy </li>
<li> Apply K Mean clustering algorithm to the data
<!--     <ul>
     <li>Extract Confusion matrix for logistic regression </li>
     <li>Extract classification report for logistic regression </li>
    </ul> -->
</li>        
</ol>
</div>

In [45]:
# setting some initial values and reading data from pickle file
pd.set_option('display.max_colwidth', -1)
color_success='#00ff66'#kickstarter green#'#2BDE73'
color_fail='#FF192F'#'#FFC7CE'#'#FFC7CE'#'#FF0000' # red
stateSuccess='Successful'
stateFail='Failed'
css = open('style/style-table.css').read() + open('style/style-notebook.css').read()
cssAnalysis = '.analysis {background:#5bffc0; }' 
HTML('<style>{} {}</style>'.format(css,cssAnalysis))
pickle_file='df_selectdata_hmda_cenus.sa'
df_selectdata = pickle.load( open( pickle_file, "rb" ) )
#df_final_months.info()
#df_final_months.head()

In [46]:
df_selectdata.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 31630026 entries, 0 to 49681
Data columns (total 21 columns):
Year                    int64
PropertyType            int64
LoanPurpose             float64
Occupancy               float64
LoanAmount              float64
ActionType              float64
MSA                     float64
StateCode               float64
CountyCode              float64
ApplicantEthnicity      float64
CoApplicantEthnicity    float64
ApplicantRace           float64
CoApplicantRace         float64
ApplicantSex            float64
CoApplicantSex          float64
ApplicantIncome         float64
PurchaserType           float64
StateName               object
CountyName              object
CLASSCODEFIPS           object
Result                  int64
dtypes: float64(15), int64(3), object(3)
memory usage: 5.2+ GB


## Helper methods

In [49]:
# Method to convert the category column into dummy columns 
def AddDummyColumnsToDataFrame(dfinput,colname,removelastdummy=False):
    print('Add {}'.format(colname))
    temp =pd.get_dummies(dfinput[colname])
    # remove one column from dummies with least value.
  
    if removelastdummy:
        t=dfinput.groupby(colname).count().state
        col_name=((t[t.values==t.min()]).index).get_values()[0]
        if col_name in temp.columns:
            print('removed column {}'.format(col_name))
            temp=temp.drop([col_name], axis=1)
    
    # remove the main column after extracting dummy
    if colname in dfinput.columns:
        print('removed column {}'.format(colname))
        dfinput =dfinput.drop([colname], axis=1)
    for col in temp:
        temp.rename(columns={col: colname+'_'+str(col)}, inplace=True)
    
    return  pd.concat([dfinput,temp], axis=1,ignore_index=False)

In [6]:
def createResultCol(row):
    if (row.ActionType==1 ):
        row.Result=1
    elif (row.ActionType==2 ):
        row.Result=0
    elif (row.ActionType==3 ):
        row.Result=-1
    elif (row.ActionType==4 ):
        row.Result=0        
    elif (row.ActionType==5 ):
        row.Result=np.NAN
    elif (row.ActionType==6 ):
        row.Result=1
    elif (row.ActionType==7 ):
        row.Result=-1
    elif (row.ActionType==8 ):
        row.Result=0        
    else:
        row.Result=np.NAN
    return row

In [5]:
# Method for applying label encoding to column
def ModifyingColumnToLabel(df,colname):
    lab_enc = preprocessing.LabelEncoder()
    _=lab_enc.fit(df[colname])
    new_val = lab_enc.transform(df[colname])
    _=df.loc[:, '{}_val'.format(colname)] = new_val

### Processing Starts

In [7]:
# filter out suspened, cancleed and live projects
#Type of Action Taken* (1,6)** Yes (2,8,4) ** No (3,7)**denied (5)*****NA
#df_selectdata_ML = pd.DataFrame()
df_selectdata_ML=df_selectdata.apply(createResultCol,axis=1)

In [51]:
df_selectdata_ML.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 31630026 entries, 0 to 49681
Data columns (total 38 columns):
Year                      int64
LoanAmount                float64
ActionType                float64
MSA                       float64
StateCode                 float64
CountyCode                float64
CoApplicantEthnicity      float64
CoApplicantRace           float64
CoApplicantSex            float64
ApplicantIncome           float64
PurchaserType             float64
StateName                 object
CountyName                object
CLASSCODEFIPS             object
Result                    int64
ApplicantEthnicity_1.0    uint8
ApplicantEthnicity_2.0    uint8
ApplicantEthnicity_3.0    uint8
ApplicantEthnicity_4.0    uint8
ApplicantRace_1.0         uint8
ApplicantRace_2.0         uint8
ApplicantRace_3.0         uint8
ApplicantRace_4.0         uint8
ApplicantRace_5.0         uint8
ApplicantRace_6.0         uint8
ApplicantRace_7.0         uint8
ApplicantSex_1.0          uint8
A

In [64]:
# Convert category columns to dummy columns=
#df_selectdata_ML=df_selectdata
categoryColumns=['StateCode','ApplicantEthnicity','ApplicantRace','ApplicantSex','Occupancy','PropertyType','LoanPurpose']
for col in categoryColumns:
    print(col)
    df_selectdata_ML=AddDummyColumnsToDataFrame(df_selectdata_ML,col)

#df_selectdata_ML.info()

StateCode
Add StateCode
removed column StateCode


In [60]:
# Add column for salary

df_selectdata_ML=df_selectdata_ML.assign(SAL_0_50=((df_selectdata_ML['ApplicantIncome']>0) & (df_selectdata_ML['ApplicantIncome']<=50)))
df_selectdata_ML=df_selectdata_ML.assign(SAL_50_100=((df_selectdata_ML['ApplicantIncome']>50) & (df_selectdata_ML['ApplicantIncome']<=100)))
df_selectdata_ML=df_selectdata_ML.assign(SAL_100_150=((df_selectdata_ML['ApplicantIncome']>100) & (df_selectdata_ML['ApplicantIncome']<=150)))
df_selectdata_ML=df_selectdata_ML.assign(SAL_150_200=((df_selectdata_ML['ApplicantIncome']>150) & (df_selectdata_ML['ApplicantIncome']<=200)))
df_selectdata_ML=df_selectdata_ML.assign(SAL_200_250=((df_selectdata_ML['ApplicantIncome']>200) & (df_selectdata_ML['ApplicantIncome']<=250)))
df_selectdata_ML=df_selectdata_ML.assign(SAL_250_300=((df_selectdata_ML['ApplicantIncome']>250) & (df_selectdata_ML['ApplicantIncome']<=300)))
df_selectdata_ML=df_selectdata_ML.assign(SAL_300_350=((df_selectdata_ML['ApplicantIncome']>300) & (df_selectdata_ML['ApplicantIncome']<=350)))
df_selectdata_ML=df_selectdata_ML.assign(SAL_350_400=((df_selectdata_ML['ApplicantIncome']>350) & (df_selectdata_ML['ApplicantIncome']<=400)))
df_selectdata_ML=df_selectdata_ML.assign(SAL_400_450=((df_selectdata_ML['ApplicantIncome']>400) & (df_selectdata_ML['ApplicantIncome']<=450)))
df_selectdata_ML=df_selectdata_ML.assign(SAL_450_500=((df_selectdata_ML['ApplicantIncome']>450) & (df_selectdata_ML['ApplicantIncome']<=500)))
df_selectdata_ML=df_selectdata_ML.assign(SAL_500_5500=((df_selectdata_ML['ApplicantIncome']>500) & (df_selectdata_ML['ApplicantIncome']<=5500)))
df_selectdata_ML=df_selectdata_ML.assign(SAL_5500_999999=((df_selectdata_ML['ApplicantIncome']>5500) & (df_selectdata_ML['ApplicantIncome']<=999999)))


In [66]:
# Add column for loan amount
df_selectdata_ML=df_selectdata_ML.assign(LonAmt_0_50=((df_selectdata_ML['LoanAmount']>0) & (df_selectdata_ML['LoanAmount']<=50)))
df_selectdata_ML=df_selectdata_ML.assign(LonAmt_50_100=((df_selectdata_ML['LoanAmount']>50) & (df_selectdata_ML['LoanAmount']<=100)))
df_selectdata_ML=df_selectdata_ML.assign(LonAmt_100_150=((df_selectdata_ML['LoanAmount']>100) & (df_selectdata_ML['LoanAmount']<=150)))
df_selectdata_ML=df_selectdata_ML.assign(LonAmt_150_200=((df_selectdata_ML['LoanAmount']>150) & (df_selectdata_ML['LoanAmount']<=200)))
df_selectdata_ML=df_selectdata_ML.assign(LonAmt_200_250=((df_selectdata_ML['LoanAmount']>200) & (df_selectdata_ML['LoanAmount']<=250)))
df_selectdata_ML=df_selectdata_ML.assign(LonAmt_250_300=((df_selectdata_ML['LoanAmount']>250) & (df_selectdata_ML['LoanAmount']<=300)))
df_selectdata_ML=df_selectdata_ML.assign(LonAmt_300_350=((df_selectdata_ML['LoanAmount']>300) & (df_selectdata_ML['LoanAmount']<=350)))
df_selectdata_ML=df_selectdata_ML.assign(LonAmt_350_400=((df_selectdata_ML['LoanAmount']>350) & (df_selectdata_ML['LoanAmount']<=400)))
df_selectdata_ML=df_selectdata_ML.assign(LonAmt_400_450=((df_selectdata_ML['LoanAmount']>400) & (df_selectdata_ML['LoanAmount']<=450)))
df_selectdata_ML=df_selectdata_ML.assign(LonAmt_450_500=((df_selectdata_ML['LoanAmount']>450) & (df_selectdata_ML['LoanAmount']<=500)))
df_selectdata_ML=df_selectdata_ML.assign(LonAmt_500_5500=((df_selectdata_ML['LoanAmount']>500) & (df_selectdata_ML['LoanAmount']<=5500)))
df_selectdata_ML=df_selectdata_ML.assign(LonAmt_5500_999999=((df_selectdata_ML['LoanAmount']>5500) & (df_selectdata_ML['LoanAmount']<=999999)))

In [67]:
df_selectdata_ML.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 31630026 entries, 0 to 49681
Columns: 114 entries, Year to LonAmt_5500_999999
dtypes: bool(24), float64(9), int64(2), object(3), uint8(76)
memory usage: 6.5+ GB


In [69]:
df_selectdata_ML_sel=df_selectdata_ML.iloc[:,13:]

In [71]:
#df_selecteddata.info()
picklefilename='df_selectdata_ML.sa'
# get the pickle file name to re confirm the previous files are not overriden.
# Note: Run this file after confimring succesful completion of all above steps
print(picklefilename)
# create pickle file for further use 
pickle.dump(df_selectdata_ML_sel,open(picklefilename,'wb'), protocol=4)

df_selectdata_ML.sa
